# Bug triage with Deep Learning - Fasttext

In [1]:
from __future__ import print_function, division

In [2]:
import fasttext
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [3]:
from methods.baseline import Baseline
from methods.experiments import Experiment
from methods.evaluation import Evaluation
from methods.retrieval import Retrieval

Using TensorFlow backend.


## Configurações Globais

In [4]:
MAX_SEQUENCE_LENGTH_T = 100 # 40
MAX_SEQUENCE_LENGTH_D = 500 # 200
EMBEDDING_DIM = 300
MAX_NB_WORDS = 20000

### Parse bugs preproprecessed

In [5]:
# Domain to use
DOMAIN = 'netbeans'
METHOD = 'fasttext'
# Dataset paths
DIR = 'data/processed/{}'.format(DOMAIN)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Path embeddings
EMBED_DIR='data/embed'
# Save model
SAVE_PATH = 'propose_feature@number_of_epochs@epochs_64batch({})'.format(DOMAIN)
SAVE_PATH_FEATURE = 'propose_feature_@number_of_epochs@epochs_64batch({})'.format(DOMAIN)

# Test bugs
PATH_TEST = 'data/processed/{}/test.txt'.format(DOMAIN)

# Extract CORPUs
EXTRACT_CORPUS = False

## Auxiliary methods

In [6]:
baseline = Baseline(DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D)
evaluation = Evaluation(verbose=0)
retrieval = Retrieval()
experiment = Experiment(baseline, evaluation)

#### Creating buckets and initialize the retrieval method

In [7]:
experiment.set_retrieval(retrieval, baseline, DOMAIN)

Creating the buckets...


#### Loading bug ids in memory

In [8]:
experiment.load_ids()
len(baseline.bug_ids)

Reading bug ids


216715

#### Dicionário de títulos e descrições

In [9]:
%%time

experiment.load_bugs()
len(baseline.sentence_dict)


CPU times: user 1min 28s, sys: 8.83 s, total: 1min 36s
Wall time: 2min 5s


#### Get buckets for each issue

In [10]:
verbose = 0
issues_by_buckets = experiment.get_buckets_for_bugs()

#### Prepare dataset

In [11]:
experiment.prepare_dataset(issues_by_buckets)

Reading train data
Reading bug ids


#### Creating the queries to test

In [12]:
# Read and create the test queries duplicate
experiment.create_queries()

Reading queries from baseline.


#### Bugs train

In [13]:
bugs_train_ids = []
for bugs in experiment.baseline.train_data:
    bugs_train_ids.append(bugs[0])
    bugs_train_ids.append(bugs[1])

#### Corpus

In [14]:
lines_title = [baseline.bug_set[bug_id]['title'] for bug_id in bugs_train_ids]
lines_desc = [baseline.bug_set[bug_id]['description'] for bug_id in bugs_train_ids]

lines = []
for title, desc in zip(lines_title, lines_desc):
    lines.append(title)
    lines.append(desc)

In [15]:
len(lines)

348728

#### Exporting corpus to .txt

In [16]:
EXPORT_PATH = os.path.join(DIR, 'corpus_fasttext_train.txt')
print(EXPORT_PATH)
with open(EXPORT_PATH, 'w') as f:
    for row in lines:
        f.write("{}\n".format(str(row).encode("utf-8")))

data/processed/netbeans/corpus_fasttext_train.txt


#### Train the method

In [17]:
%%time

import os

cpu = os.cpu_count() - 1

# Skipgram model :
model = fasttext.train_unsupervised(EXPORT_PATH, 
                                    model='cbow', 
                                    minCount=1,
                                    neg=10,
                                    wordNgrams=3,
                                    epoch=15, 
                                    dim=300, 
                                    loss='hs', 
                                    thread=cpu, 
                                    verbose=2)

CPU times: user 6h 8min 33s, sys: 9.1 s, total: 6h 8min 42s
Wall time: 52min 46s


#### Rank@25

In [18]:
recall, exported_rank = experiment.evaluate_validation_test(retrieval, verbose, model, issues_by_buckets, method=METHOD)
"recall@25 last epoch:", recall

('recall@25 last epoch:', 0.24)

### Retrieval evaluation

In [19]:
print("Total of queries:", len(retrieval.test))

Total of queries: 7164


In [20]:
exported_rank[:20]

['198406:196608|196608:0.72719606757164,220410:0.6043229699134827,168190:0.6016494631767273,192420:0.5879468619823456,189916:0.5827312171459198,157361:0.5816790163516998,160273:0.579177588224411,147188:0.5725648701190948,197248:0.5660541355609894,182992:0.5631988048553467,219447:0.562169075012207,218475:0.558842808008194,201296:0.5577436089515686,144166:0.5553570687770844,20384:0.5500480234622955,145969:0.5494462251663208,129227:0.548858255147934,160099:0.5482386946678162,142725:0.5480367541313171,219832:0.5475067496299744,174450:0.5459086000919342,188485:0.5425573587417603,236095:0.5412673056125641,178640:0.5395246148109436,10422:0.538762241601944',
 '189728:198719|191620:0.606585294008255,182625:0.5970258414745331,183595:0.5618892014026642,182643:0.5599482357501984,200469:0.551433652639389,182504:0.5302617847919464,202588:0.5126890242099762,218601:0.509366899728775,225451:0.5009149610996246,177914:0.49571549892425537,222191:0.484694242477417,202385:0.4796716570854187,194488:0.4780075

#### Exporting the result

In [21]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))

In [22]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [23]:
report = experiment.evaluation.evaluate(EXPORT_RANK_PATH)
report

{'1 - recall_at_5': 0.2,
 '2 - recall_at_10': 0.21,
 '3 - recall_at_15': 0.23,
 '4 - recall_at_20': 0.24,
 '5 - recall_at_25': 0.24}